# Preprocessing BUSI dataset

# Important libraries

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import glob
from scipy import ndimage

# Preprocessing class with all needed functions

In [2]:
class BUSIPreprocessor:
    def __init__(self, data_dir, output_dir, img_size=(224, 224)):
        """
        Initialize the BUSI dataset preprocessor.
        
        Args:
            data_dir: Directory containing the BUSI dataset with benign, malignant, normal folders
            output_dir: Directory to save processed images
            img_size: Target size for the processed images
        """
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.img_size = img_size
        
        # Create output directories if they don't exist
        for class_name in ['benign', 'malignant', 'normal']:
            os.makedirs(os.path.join(output_dir, "processed_images", class_name), exist_ok=True)
        
        # Define class mapping
        self.class_mapping = {
            'normal': 0,
            'benign': 1,
            'malignant': 2
        }
        
        
    # loading an image and grayscaling it if necessary
    def load_image(self, image_path):
        """Load an image and convert to grayscale if needed."""
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Could not load image: {image_path}")
        
        # Convert to grayscale if it's a color image
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        return img

    

    # Annotations removal (letters, texts, drawings) & top left corner annotation removal
    def add_corner_triangle_mask(self, img):
        """
        Adds a black triangle mask to the left corner of the image
        """
        height, width = img.shape[:2]
        
        # Create a copy of the input image
        result = img.copy()
        
        # Define triangle vertices (adjust these coordinates as needed)
        # Format: top-left corner, bottom-left corner, and a point to the right
        triangle_pts = np.array([[0, 0], [0, height//16], [width//16, 0]], np.int32)
        
        # Fill the triangle with black (0)
        cv2.fillPoly(result, [triangle_pts], 0)
        
        return result

    def remove_annotations(self, img):
        # Threshold for bright pixels
        _, mask = cv2.threshold(img, 220, 255, cv2.THRESH_BINARY)
    
        # Morphological operations to remove thin text/crosses
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        mask = cv2.dilate(mask, kernel, iterations=1)
    
        # Inpaint
        inpainted = cv2.inpaint(img, mask, inpaintRadius=1, flags=cv2.INPAINT_TELEA)
    
        return inpainted


        
    # Filters applied to the BUSI images
    def enhance_contrast(self, img):
        """
        Enhance image contrast using adaptive CLAHE,
        tuned for ultrasound images.
        """
        if img.dtype != np.uint8:
            img = (img * 255).astype(np.uint8)
    
        # Adaptive CLAHE parameters for ultrasound images
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(img)
        return enhanced
    
    def normalize_image(self, img):
        """
        Normalize image to [0, 1] range.
        """
        img = img.astype(np.float32)
        
        # Normalize to 0-1 range
        img_min = np.min(img)
        img_max = np.max(img)
        
        if img_max > img_min:
            img = (img - img_min) / (img_max - img_min)
        
        return img
    


    # Resizing to a standard form, to help avoid CNN bias
    def resize_with_padding(self, img, target_size):
        h, w = img.shape[:2]
        target_h, target_w = target_size
    
        # Compute scale and new size
        scale = min(target_w / w, target_h / h)
        new_w, new_h = int(w * scale), int(h * scale)
    
        # Resize while preserving aspect ratio
        resized_img = cv2.resize(img, (new_w, new_h))
    
        # Create a black canvas
        padded_img = np.zeros((target_h, target_w), dtype=resized_img.dtype)
    
        # Compute padding offsets
        x_offset = (target_w - new_w) // 2
        y_offset = (target_h - new_h) // 2
    
        # Place the resized image on the canvas
        padded_img[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized_img
    
        return padded_img



        
    # All steps in processing an image (the order is important for efficiency and reliability)
    def process_image(self, image_path, class_name, save=True):
        """Process a single ultrasound image."""
    
        # Extract the image filename
        filename = os.path.basename(image_path)
        name_without_ext = os.path.splitext(filename)[0]
        
        # Load the image
        original_img = self.load_image(image_path)

        #  Top left corner annotation removal & annotations removal (letters, texts, drawings)
        img_left_corner_removed = self.add_corner_triangle_mask(original_img)
        img_reduced = self.remove_annotations(img_left_corner_removed)
        
        # Filters
        img_enhanced = self.enhance_contrast(img_reduced) # Enhance contrast
        img_normalized = self.normalize_image(img_enhanced) # Normalize the image
        
        # Resize the image to the target size (224,224) -> useful for the CNN 
        img_resized = self.resize_with_padding(img_enhanced, self.img_size)
        
        # Save the processed image
        if save:
            output_path = os.path.join(self.output_dir, "processed_images", class_name, 
                                     f"{name_without_ext}_processed.png")
            cv2.imwrite(output_path, img_resized)
        
        return {
            'filename': filename,
            'class': class_name,
            'processed_image': img_resized,
            'normalized_image': self.resize_with_padding((img_normalized * 255).astype(np.uint8), self.img_size)
        }

    # For mask, only resing is needed    
    def process_mask_image(self, mask_path, class_name, save=True):
        """Process a mask image without any modifications."""
        # Extract the mask filename
        filename = os.path.basename(mask_path)
        name_without_ext = os.path.splitext(filename)[0]
        
        # Load the mask image
        mask_img = self.load_image(mask_path)
        
        # Only resize the mask to match the target size, no other processing
        mask_resized = self.resize_with_padding(mask_img, self.img_size)
        
        # Save the mask image
        if save:
            output_path = os.path.join(self.output_dir, "processed_images", class_name, filename)
            cv2.imwrite(output_path, mask_resized)
        
        return {
            'filename': filename,
            'class': class_name,
            'mask_image': mask_resized
        }

        
    
    def get_image_files(self, class_folder):
        """
        Get all image files from a class folder, excluding mask files.
        Returns both regular images and their corresponding mask files (if they exist).
        """
        image_extensions = ['*.png', '*.jpg', '*.jpeg', '*.bmp', '*.tiff']
        image_files = []
        mask_files = []
        
        for ext in image_extensions:
            files = glob.glob(os.path.join(class_folder, ext))
            
            # Separate regular images and mask images
            for f in files:
                if '_mask' in os.path.basename(f).lower():
                    mask_files.append(f)
                else:
                    image_files.append(f)
        
        return image_files, mask_files


    def process_all_images(self):
        """Process all images and their masks in the dataset."""
        image_results = []
        mask_results = []
        
        for class_name in ['malignant', 'normal', 'benign']:
            class_folder = os.path.join(self.data_dir, class_name)
            
            if not os.path.exists(class_folder):
                print(f"Warning: Folder {class_folder} does not exist. Skipping...")
                continue
            
            # Get both regular images and mask images
            image_files, mask_files = self.get_image_files(class_folder)
            print(f"Found {len(image_files)} images and {len(mask_files)} masks in {class_name} folder")
            
            # Process regular images
            for image_path in tqdm(image_files, desc=f"Processing {class_name} images"):
                try:
                    result = self.process_image(image_path, class_name)
                    image_results.append(result)
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")
            
            # Process mask images (without modifications)
            for mask_path in tqdm(mask_files, desc=f"Processing {class_name} masks"):
                try:
                    result = self.process_mask_image(mask_path, class_name)
                    mask_results.append(result)
                except Exception as e:
                    print(f"Error processing mask {mask_path}: {e}")
        
        return image_results, mask_results

    
    def create_dataset(self, use_normalized=True, combine_masks=True):
        """Create a dataset for training a CNN."""
        image_data, mask_data = self.process_all_images()
    
        # Create dictionaries for easy matching - supports multiple masks per image
        mask_dict = {}
        for mask in mask_data:
            # Extract base filename without _mask suffix and any additional suffixes like _1
            mask_filename = mask['filename']
            # Remove file extension first
            base_name = os.path.splitext(mask_filename)[0]
            # Remove _mask and any additional suffixes like _1, _2, etc.
            if '_mask' in base_name:
                base_name = base_name.split('_mask')[0]
            
            # Store multiple masks per image in a list
            if base_name not in mask_dict:
                mask_dict[base_name] = []
            mask_dict[base_name].append(mask['mask_image'])
        
        # Create X (images), y (labels), and masks
        X = []
        y = []
        masks = []
        filenames = []
        
        for data in image_data:
            # Extract base name from the current image
            img_filename = data['filename']
            img_base_name = os.path.splitext(img_filename)[0]
            
            if use_normalized:
                X.append(data['normalized_image'])
            else:
                X.append(data['processed_image'])
            
            y.append(self.class_mapping[data['class']])
            
            # Get corresponding masks
            if img_base_name in mask_dict:
                img_masks = mask_dict[img_base_name]
                
                if len(img_masks) == 1:
                    masks.append(img_masks[0])
                elif combine_masks:
                    # Combine multiple masks by taking the maximum value at each pixel
                    combined_mask = np.maximum.reduce(img_masks)
                    masks.append(combined_mask)
                    print(f"Combined {len(img_masks)} masks for {img_filename}")
                else:
                    # Just use the first mask if not combining
                    masks.append(img_masks[0])
                    print(f"Using first mask out of {len(img_masks)} for {img_filename}")
            else:
                # If no mask found, create a blank mask
                blank_mask = np.zeros(self.img_size, dtype=np.uint8)
                masks.append(blank_mask)
                print(f"Warning: No mask found for {img_filename}, using blank mask")
            
            filenames.append(data['filename'])
        
        # Convert to numpy arrays
        X = np.array(X)
        y = np.array(y)
        masks = np.array(masks)
    
        # Add channel dimension if needed (for CNN)
        if len(X.shape) == 3:
            X = np.expand_dims(X, axis=-1)
        if len(masks.shape) == 3:
            masks = np.expand_dims(masks, axis=-1)
        
        print(f"Dataset shape: {X.shape}")
        print(f"Masks shape: {masks.shape}")
        print(f"Labels shape: {y.shape}")
        print(f"Class distribution: Normal: {np.sum(y == 0)}, Benign: {np.sum(y == 1)}, Malignant: {np.sum(y == 2)}")
        
        # Split into training and validation sets
        X_train, X_val, y_train, y_val, masks_train, masks_val = train_test_split(
            X, y, masks, test_size=0.2, random_state=42, stratify=y
        )
        
        return X_train, X_val, y_train, y_val, masks_train, masks_val, filenames


# Data augmentation

In [3]:
def data_augmentation(X_train, y_train, masks=None):
    """
    Apply data augmentation to the training set (supports optional masks).
    
    Args:
        X_train: Training images
        y_train: Training labels (0=Normal, 1=Benign, 2=Malignant)
        masks: Optional masks (None if classification only)
        
    Returns:
        Augmented images, labels, (and masks if provided)
    """
    X_augmented, y_augmented = [], []
    mask_augmented = [] if masks is not None else None

    for i in range(len(X_train)):
        img = X_train[i]
        label = y_train[i]
        mask = masks[i] if masks is not None else None

        # Define per-class augmentation strength
        if label == 0:   # Normal (minority)
            aug_factor = 4
        elif label == 2: # Malignant (mid-size)
            aug_factor = 3
        else:            # Benign (majority)
            aug_factor = 1

        # Always keep original
        X_augmented.append(img)
        y_augmented.append(label)
        if mask is not None:
            mask_augmented.append(mask)

        # Augmentations
        for _ in range(aug_factor):
            aug_type = np.random.choice(['flip', 'rotate', 'zoom', 'noise', 'brightness'])

            img_2d = img.squeeze(-1) if img.ndim == 3 and img.shape[-1] == 1 else img
            mask_2d = mask.squeeze(-1) if (mask is not None and mask.ndim == 3 and mask.shape[-1] == 1) else mask

            if aug_type == 'flip':
                aug_img = cv2.flip(img_2d, 1)
                aug_mask = cv2.flip(mask_2d, 1) if mask is not None else None

            elif aug_type == 'rotate':
                angle = np.random.uniform(-15, 15)
                aug_img = ndimage.rotate(img_2d, angle, reshape=False)
                aug_mask = ndimage.rotate(mask_2d, angle, reshape=False) if mask is not None else None

            elif aug_type == 'zoom':
                zoom_factor = np.random.uniform(0.9, 1.1)
                h, w = img_2d.shape
                new_h, new_w = int(h * zoom_factor), int(w * zoom_factor)
                aug_img = cv2.resize(img_2d, (new_w, new_h))
                aug_mask = cv2.resize(mask_2d, (new_w, new_h)) if mask is not None else None

                if zoom_factor < 1.0:  # pad
                    pad_h = (h - new_h) // 2
                    pad_w = (w - new_w) // 2
                    aug_img = cv2.copyMakeBorder(aug_img, pad_h, h - new_h - pad_h,
                                                 pad_w, w - new_w - pad_w,
                                                 cv2.BORDER_CONSTANT, value=0)
                    if mask is not None:
                        aug_mask = cv2.copyMakeBorder(aug_mask, pad_h, h - new_h - pad_h,
                                                      pad_w, w - new_w - pad_w,
                                                      cv2.BORDER_CONSTANT, value=0)
                else:  # crop
                    start_h = (new_h - h) // 2
                    start_w = (new_w - w) // 2
                    aug_img = aug_img[start_h:start_h + h, start_w:start_w + w]
                    if mask is not None:
                        aug_mask = aug_mask[start_h:start_h + h, start_w:start_w + w]

            elif aug_type == 'noise':
                noise = np.random.normal(0, np.random.uniform(3, 8), img_2d.shape)
                aug_img = np.clip(img_2d.astype(np.float32) + noise, 0, 255).astype(np.uint8)
                aug_mask = mask_2d

            elif aug_type == 'brightness':
                factor = np.random.uniform(0.9, 1.1)
                aug_img = np.clip(img_2d.astype(np.float32) * factor, 0, 255).astype(np.uint8)
                aug_mask = mask_2d

            # Restore channel dim
            if img.ndim == 3 and img.shape[-1] == 1:
                aug_img = np.expand_dims(aug_img, axis=-1)
                if aug_mask is not None:
                    aug_mask = np.expand_dims(aug_mask, axis=-1)

            X_augmented.append(aug_img)
            y_augmented.append(label)
            if mask is not None:
                mask_augmented.append(aug_mask)

    if masks is not None:
        return np.array(X_augmented), np.array(y_augmented), np.array(mask_augmented)
    else:
        return np.array(X_augmented), np.array(y_augmented)


# Creating the instance of the BUSI processing class, applying preprocessing, creating dataset for CNN and applying data augmentation

In [4]:
data_dir = "C:/Users/DragosTrandafiri/BreastCancer_CNN/data/raw/busi"   # benign, malignant, normal folders
output_dir = "C:/Users/DragosTrandafiri/BreastCancer_CNN/data/processed/busi"

# Create preprocessor
preprocessor = BUSIPreprocessor(data_dir, output_dir, img_size=(224, 224))

# Create dataset
X_train, X_val, y_train, y_val, masks_train, masks_val, filenames = preprocessor.create_dataset()

# Apply data augmentation with masks
X_train_aug, y_train_aug, masks_train_aug = data_augmentation(X_train, y_train, masks=masks_train)

print(f"Original training set: {X_train.shape}, {y_train.shape}, masks: {masks_train.shape}")
print(f"Augmented training set: {X_train_aug.shape}, {y_train_aug.shape}, masks: {masks_train_aug.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}, masks: {masks_val.shape}")

# Print class distribution
unique, counts = np.unique(y_train_aug, return_counts=True)
class_names = ['Normal', 'Benign', 'Malignant']
for i, (class_idx, count) in enumerate(zip(unique, counts)):
    print(f"Augmented {class_names[class_idx]}: {count} samples")

Found 210 images and 211 masks in malignant folder


Processing malignant masks: 100%|███████████████████████████████████████████████████| 211/211 [00:00<00:00, 698.65it/s]


Found 133 images and 133 masks in normal folder


Processing normal masks: 100%|██████████████████████████████████████████████████████| 133/133 [00:00<00:00, 307.68it/s]


Found 437 images and 454 masks in benign folder


Processing benign masks: 100%|██████████████████████████████████████████████████████| 454/454 [00:00<00:00, 654.58it/s]


Combined 2 masks for malignant (53).png
Combined 2 masks for benign (100).png
Combined 2 masks for benign (163).png
Combined 2 masks for benign (173).png
Combined 2 masks for benign (181).png
Combined 3 masks for benign (195).png
Combined 2 masks for benign (25).png
Combined 2 masks for benign (315).png
Combined 2 masks for benign (346).png
Combined 2 masks for benign (4).png
Combined 2 masks for benign (424).png
Combined 2 masks for benign (54).png
Combined 2 masks for benign (58).png
Combined 2 masks for benign (83).png
Combined 2 masks for benign (92).png
Combined 2 masks for benign (93).png
Combined 2 masks for benign (98).png
Dataset shape: (780, 224, 224, 1)
Masks shape: (780, 224, 224, 1)
Labels shape: (780,)
Class distribution: Normal: 133, Benign: 437, Malignant: 210
Original training set: (624, 224, 224, 1), (624,), masks: (624, 224, 224, 1)
Augmented training set: (1902, 224, 224, 1), (1902,), masks: (1902, 224, 224, 1)
Validation set: (156, 224, 224, 1), (156,), masks: (156

In [13]:
for i in range (0, 50):
    print(X_train_aug[i][22][22])
    print(y_train_aug[i])
    print(masks_train_aug[i][22][22])
    print("------------------------")
    print("                        ")

[242]
1
[0]
------------------------
                        
[0]
1
[0]
------------------------
                        
[0]
1
[0]
------------------------
                        
[0]
1
[0]
------------------------
                        
[176]
2
[0]
------------------------
                        
[162]
2
[0]
------------------------
                        
[172]
2
[0]
------------------------
                        
[165]
2
[0]
------------------------
                        
[231]
1
[0]
------------------------
                        
[0]
1
[0]
------------------------
                        
[190]
1
[0]
------------------------
                        
[190]
1
[0]
------------------------
                        
[194]
1
[0]
------------------------
                        
[0]
1
[0]
------------------------
                        
[0]
2
[0]
------------------------
                        
[0]
2
[0]
------------------------
                        
[0]
2
[0]
------------

In [18]:
import os
import cv2
import numpy as np

# Define output directory for augmented images
augmented_dir_X = "C:/Users/DragosTrandafiri/BreastCancer_CNN/data/augmented/busi/X"
augmented_dir_Mask = "C:/Users/DragosTrandafiri/BreastCancer_CNN/data/augmented/busi/Mask"

# Class names mapping
class_names = ['Normal', 'Benign', 'Malignant']

# Create directories if they don’t exist
for class_name in class_names:
    os.makedirs(os.path.join(augmented_dir_X, class_name), exist_ok=True)
    os.makedirs(os.path.join(augmented_dir_Mask, class_name), exist_ok=True)

# Save images
for i, (img_X, img_Mask, label) in enumerate(zip(X_train_aug, masks_train_aug, y_train_aug)):
    # Convert from float [0,1] to uint8 [0,255] if needed
    if img_X.max() <= 1.0:
        img_to_save_X = (img_X * 255).astype(np.uint8)
    else:
        img_to_save_X = img_X.astype(np.uint8)

    if img_Mask.max() <= 1.0:
        img_to_save_mask = (img_Mask * 255).astype(np.uint8)
    else:
        img_to_save_mask = img_Mask.astype(np.uint8)

    # Ensure channel order is correct (OpenCV uses BGR)
    if img_to_save_X.shape[-1] == 3:  
        img_to_save_X = cv2.cvtColor(img_to_save_X, cv2.COLOR_RGB2BGR)

    if img_to_save_mask.shape[-1] == 3:  
        img_to_save_mask = cv2.cvtColor(img_to_save_mask, cv2.COLOR_RGB2BGR)
    

    # Build filename
    filename = f"aug_{i:05d}.png"
    filepath_x = os.path.join(augmented_dir_X, class_names[label], filename)

    # Save
    cv2.imwrite(filepath_x, img_to_save_X)


    filename = f"aug_mask_{i:05d}.png"
    filepath_mask = os.path.join(augmented_dir_Mask, class_names[label], filename)

    # Save
    cv2.imwrite(filepath_mask, img_to_save_mask)

print(f"Augmented images saved to: {augmented_dir_X}")



Augmented images saved to: C:/Users/DragosTrandafiri/BreastCancer_CNN/data/augmented/busi/X


In [5]:
augmented_dir_X_non_aug = "C:/Users/DragosTrandafiri/BreastCancer_CNN/data/non_augmented/busi/X"
augmented_dir_Mask_non_aug = "C:/Users/DragosTrandafiri/BreastCancer_CNN/data/non_augmented/busi/Mask"

# Class names mapping
class_names = ['Normal', 'Benign', 'Malignant']

# Create directories if they don’t exist
for class_name in class_names:
    os.makedirs(os.path.join(augmented_dir_X_non_aug, class_name), exist_ok=True)
    os.makedirs(os.path.join(augmented_dir_Mask_non_aug, class_name), exist_ok=True)

for i, (img_X, img_Mask, label) in enumerate(zip(X_train, masks_train, y_train)):
    # Convert from float [0,1] to uint8 [0,255] if needed
    if img_X.max() <= 1.0:
        img_to_save_X = (img_X * 255).astype(np.uint8)
    else:
        img_to_save_X = img_X.astype(np.uint8)

    if img_Mask.max() <= 1.0:
        img_to_save_mask = (img_Mask * 255).astype(np.uint8)
    else:
        img_to_save_mask = img_Mask.astype(np.uint8)

    # Ensure channel order is correct (OpenCV uses BGR)
    if img_to_save_X.shape[-1] == 3:  
        img_to_save_X = cv2.cvtColor(img_to_save_X, cv2.COLOR_RGB2BGR)

    if img_to_save_mask.shape[-1] == 3:  
        img_to_save_mask = cv2.cvtColor(img_to_save_mask, cv2.COLOR_RGB2BGR)
    

    # Build filename
    filename = f"aug_{i:05d}.png"
    filepath_x = os.path.join(augmented_dir_X_non_aug, class_names[label], filename)

    # Save
    cv2.imwrite(filepath_x, img_to_save_X)


    filename = f"aug_mask_{i:05d}.png"
    filepath_mask = os.path.join(augmented_dir_Mask_non_aug, class_names[label], filename)

    # Save
    cv2.imwrite(filepath_mask, img_to_save_mask)
